In [ ]:
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow as tf
import keras_tuner as kt
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import label_binarize
from sklearn.metrics import roc_auc_score
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import precision_recall_fscore_support
from itertools import cycle

In [ ]:
n_classes = 3
num_classes = 3
split = 44
n=3

vIDs = ['CRC01STB-TS','CRC01STC-TS','CRC01SUB-TS','CRC01SUC-TS','CRC01SUP-TS','CRC02STA-TS',
        'CRC02STB-TS','CRC02STC-TS','CRC02SUA-TS','CRC02SUB-TS','CRC02SUC-TS','CRC03STC-TS',
        'CRC03STP-TS','CRC03SUB-TS','CRC03SUC-TS','CRC03SUP-TS','CRC04STA-TS','CRC04STB-TS',
        'CRC04STC-TS','CRC04SUA-TS','CRC04SUB-TS','CRC04SUC-TS','CRC05STB-TS','CRC05STC-TS',
        'CRC05STP-TS','CRC05SUB-TS','CRC05SUC-TS','CRC05SUP-TS','CRC06STA-TS','CRC06STB-TS',
        'CRC06STC-TS','CRC06SUA-TS','CRC06SUB-TS','CRC06SUC-TS','CRC07STA-TS','CRC07STB-TS',
        'CRC07STC-TS','CRC07SUA-TS','CRC07SUB-TS','CRC07SUC-TS','CRC08STB-TS','CRC08STC-TS',
        'CRC08STP-TS','CRC08SUB-TS','CRC08SUC-TS','CRC08SUP-TS','CRC09STA-TS','CRC09STB-TS',
        'CRC09STC-TS','CRC09SUA-TS','CRC09SUB-TS','CRC09SUC-TS','CRC10STB-TS','CRC10STC-TS',
        'CRC10STP-TS','CRC10SUB-TS','CRC10SUC-TS','CRC10SUP-TS','CRC11STA-TS','CRC11STB-TS',
        'CRC11STC-TS','CRC11SUA-TS','CRC11SUB-TS','CRC11SUC-TS','CRC12STB-TS','CRC12STC-TS',
        'CRC12STP-TS','CRC12SUB-TS','CRC12SUC-TS','CRC12SUP-TS','CRC13STA-TS','CRC13STB-TS',
        'CRC13STC-TS','CRC13SUA-TS','CRC13SUB-TS','CRC13SUC-TS','CRC14STB-TS','CRC14STC-TS',
        'CRC14STP-TS','CRC14SUB-TS','CRC14SUC-TS','CRC14SUP-TS']


In [ ]:
ind = [ 0 , 6, 17 ,22 ,29 ,35 ,40 ,47, 52 ,59, 64 ,71 ,76  ,2  ,9 ,13, 20 ,25 ,32 ,38, 43 ,50 ,55 ,62, 67, 74, 79]

j = 0
vRR = []
for ID in vIDs:
    for x in ind:
        if(x==j):
            flag=1
            break
        else:
            flag=0
    if(flag==0):
        df = pd.read_csv('Data/CRC_FC/'+ID+'.csv')
        nw = df['RRts'].to_numpy()
        # nw = (nw-np.mean(nw))/np.std(nw)
        new = nw[:356]
        vRR.append(new)
    j=j+1
    


x = np.asarray(vRR)
xMean = np.mean(x)
xStd = np.std(x)
x = (x-np.mean(x))/np.std(x)
x = x.reshape((x.shape[0], x.shape[1], 1))

df_y = pd.read_csv('Data/CRC/Labels.csv') 

state_abc = df_y.loc[:,'condition']

condition2 = np.zeros(83)
condition2[state_abc == 'STA'] = 1 # bloqueo parasimpatico
condition2[state_abc == 'STB'] = 0 # bloqueo ambos
condition2[state_abc == 'STC'] = 0 # sin bloqueo
condition2[state_abc == 'STP'] = 2 # bloqueo simpatico
condition2[state_abc == 'SUA'] = 1 # bloqueo parasimpatico
condition2[state_abc == 'SUB'] = 0 # bloqueo ambos
condition2[state_abc == 'SUC'] = 0 # sin bloqueo
condition2[state_abc == 'SUP'] = 2 # bloqueo simpatico

y_new = condition2[0:82]


index_1 = np.where(state_abc == 'STB')[0]
index_2 = np.where(state_abc == 'SUB')[0]

index = np.concatenate((index_1,index_2))

y = np.delete(y_new, index)


skf = StratifiedKFold(n_splits=5)


y_train = y[:split]
y_test = y[split:]

x_train = x[:split]
x_test = x[split:]


print(y_test)
print(xMean,xStd)


In [ ]:
skf.get_n_splits(x, y)

vec_y_train =[]
vec_y_test =[]
vec_x_train =[]
vec_x_test =[]

for train_index, test_index in skf.split(x, y):
    x_train, x_test = x[train_index], x[test_index]
    y_train, y_test = y[train_index], y[test_index]
    vec_y_train.append(y_train)
    vec_y_test.append(y_test)
    vec_x_train.append(x_train)
    vec_x_test.append(x_test)
    

In [ ]:
print(vec_y_test[3])

In [ ]:
size = 3
act = 'ReLU'


def make_model(input_shape):
    input_layer = keras.layers.Input(input_shape)
    
    #conv1 = keras.layers.Conv1D(filters=512, kernel_size=size, padding="same", activation=act)(input_layer)      
    #conv1 = keras.layers.BatchNormalization()(conv1)    
    
    conv2 = keras.layers.Conv1D(filters=256, kernel_size=size, padding="same", activation=act)(input_layer)      
    conv2 = keras.layers.BatchNormalization()(conv2)
    
    conv3 = keras.layers.Conv1D(filters=128, kernel_size=size, padding="same", activation=act)(conv2)       
    conv3 = keras.layers.BatchNormalization()(conv3)

    conv4 = keras.layers.Conv1D(filters=64, kernel_size=size, padding="same", activation=act)(conv3)
    conv4 = keras.layers.BatchNormalization()(conv4)

    conv5 = keras.layers.Conv1D(filters=32, kernel_size=size, padding="same", activation=act)(conv4)    
    conv5 = keras.layers.BatchNormalization()(conv5)   
    
#     conv6 = keras.layers.Conv1D(filters=16, kernel_size=size, padding="same", activation=act)(conv5)    
#     conv6 = keras.layers.BatchNormalization()(conv6) 
    
    do = tf.keras.layers.Dropout(.2)(conv5)
    gap = keras.layers.GlobalMaxPooling1D()(do) 

    output_layer = keras.layers.Dense(num_classes, activation="softmax")(gap)

    return keras.models.Model(inputs=input_layer, outputs=output_layer)


model = make_model(input_shape=x_train.shape[1:])
keras.utils.plot_model(model, show_shapes=True)

In [ ]:
# model = keras.models.load_model("Models/Model_FC_356.h5", compile = False)
# keras.utils.plot_model(model, show_shapes=True)

In [ ]:
epochs = 1000
batch_size = 64

callbacks = [
    keras.callbacks.ModelCheckpoint(
        "Model_FC_356_f3.h5", save_best_only=True, monitor="val_sparse_categorical_accuracy"
    ),
    keras.callbacks.ReduceLROnPlateau(
        monitor="val_loss", factor=0.5, patience=20, min_lr=0.000005
    ),
    keras.callbacks.EarlyStopping(monitor="val_loss", patience=50, verbose=1),
]

model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["sparse_categorical_accuracy"],
)

n=3

history = model.fit(
    vec_x_train[n],
    vec_y_train[n],
    batch_size=batch_size,
    epochs=epochs,
    callbacks=callbacks,
    validation_data=(vec_x_test[n], vec_y_test[n]),
    verbose=1,
)


In [ ]:
model.summary()

In [ ]:
train = history.history[metric]
val = history.history["val_" + metric]

In [ ]:
# metric = "sparse_categorical_accuracy"
metric = "loss"


# train_full = np.concatenate((train_full,train))
# train_val = np.concatenate((train_val,val))

train_full = history.history[metric]
train_val = history.history["val_" + metric]

plt.rcParams["figure.dpi"] = 120
plt.rcParams["figure.figsize"] = (6.96,6.96)
plt.rcParams.update({'font.size': 18})
plt.rcParams['axes.labelsize'] = 18
plt.rcParams['axes.labelweight'] = 'bold'

# metric = "sparse_categorical_accuracy"

plt.figure()
# plt.figure(figsize=(1, 1))
plt.plot(train_full)
plt.plot(train_val)
# plt.title("model " + metric)
plt.ylabel("Accuracy", fontsize="large")
plt.xlabel("Epoch", fontsize="large")
# plt.xlim([-10, 1000])

plt.ylim([0.0, 1.05])

plt.legend(["Train", "Val"], loc="best")
# plt.show()
plt.tight_layout()
plt.savefig(metric)

In [ ]:
model = keras.models.load_model("Models/Model_FC_356_f3.h5", compile = False)
# keras.utils.plot_model(model, show_shapes=True)

model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["sparse_categorical_accuracy"],
)
keras.utils.plot_model(model, show_shapes=True)

In [ ]:
test_loss, test_acc = model.evaluate(vec_x_test[n], vec_y_test[n])
y_test_b = vec_y_test[n]
print("Test accuracy", test_acc)
print("Test loss", test_loss)
# print(history.history)

In [ ]:
skf.get_n_splits(x, y)

vec_y_train =[]
vec_y_test =[]
vec_x_train =[]
vec_x_test =[]

for train_index, test_index in skf.split(x, y):
    x_train, x_test = x[train_index], x[test_index]
    y_train, y_test = y[train_index], y[test_index]
    vec_y_train.append(y_train)
    vec_y_test.append(y_test)
    vec_x_train.append(x_train)
    vec_x_test.append(x_test)
    

In [ ]:
y_score = model.predict(vec_x_test[n])
y_test = vec_y_test[n]

print(y_score)
# print(y_score[:,0])
# y_test[y_test==2] = 1
print(y_test)
y_true = y_test
# fpr,tpr,th= roc_curve(vec_y_test[n], y)

In [ ]:
# for score in y_score:
#     if(score[0] > score[1] and score[0] > score[2]):
#         score[0] = 1
#         score[1] = 0
#         score[2] = 0
#     if(score[1] > score[0] and score[1] > score[2]):
#         score[0] = 0
#         score[1] = 1
#         score[2] = 0
#     if(score[2] > score[0] and score[2] > score[1]):
#         score[0] = 0
#         score[1] = 0
#         score[2] = 1

print(y_score)

In [ ]:
y_test = label_binarize(y_test, classes=[0, 1, 2])
print(y_test)

In [ ]:
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_test[:, i], y_score[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

In [ ]:
# First aggregate all false positive rates
all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))

# Then interpolate all ROC curves at this points
mean_tpr = np.zeros_like(all_fpr)
for i in range(n_classes):
    mean_tpr += np.interp(all_fpr, fpr[i], tpr[i])

# Finally average it and compute AUC
mean_tpr /= n_classes

fpr["macro"] = all_fpr
tpr["macro"] = mean_tpr
roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])

# Plot all ROC curves
plt.figure()
lw = 2
# plt.plot(
#     fpr["micro"],
#     tpr["micro"],
#     label="micro-average ROC curve (area = {0:0.2f})".format(roc_auc["micro"]),
#     color="deeppink",
#     linestyle=":",
#     linewidth=4,
# )

plt.plot(
    fpr["macro"],
    tpr["macro"],
    label="macro-average ROC curve (area = {0:0.2f})".format(roc_auc["macro"]),
    color="navy",
    linestyle=":",
    linewidth=4,
)

colors = cycle(["aqua", "darkorange", "cornflowerblue"])
for i, color in zip(range(n_classes), colors):
    plt.plot(
        fpr[i],
        tpr[i],
        color=color,
        lw=lw,
        label="ROC curve of class {0} (area = {1:0.2f})".format(i, roc_auc[i]),
    )

plt.plot([0, 1], [0, 1], "k--", lw=lw)
plt.xlim([-0.05, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("Some extension of Receiver operating characteristic to multiclass")
plt.legend(loc="lower right")
plt.show()


roc_x = np.concatenate(([0],fpr["macro"]))
roc_y = np.concatenate(([0],tpr["macro"]))


plt.figure()
plt.plot(roc_x,roc_y,label='ROC_fold3',linestyle=":")
plt.plot([0, 1], [0, 1], "k--", lw=lw)
plt.axis('square')
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.legend(loc="lower right")

In [ ]:
fpr,tpr,th= roc_curve(y_test[:,0], y_score[:,0])
roc_auc = auc(fpr, tpr)



# nn_fpr_keras, nn_tpr_keras, nn_thresholds_keras = roc_curve(y_test, classes)
# auc_keras = auc(nn_fpr_keras, nn_tpr_keras)
# print(auc_keras)
plt.rcParams["figure.dpi"] = 120
plt.rcParams["figure.figsize"] = (11.458,6.96)
plt.rcParams.update({'font.size': 18})
plt.rcParams['axes.labelsize'] = 18
plt.rcParams['axes.labelweight'] = 'bold'

plt.figure()
plt.axis('square')
lw = 2
plt.plot(
    fpr,
    tpr,
    color="darkorange",
    lw=lw,
    label="ROC curve (area = %0.2f)" % roc_auc,
)
plt.plot([0, 1], [0, 1], color="navy", lw=lw, linestyle="--")
plt.xlim([-0.05, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("Receiver operating characteristic")
plt.legend(loc="lower right")
plt.tight_layout()
plt.savefig('ROC')

# plt.rcParams["figure.dpi"] = 120
# plt.rcParams["figure.figsize"] = (11.458,6.96)
# plt.rcParams.update({'font.size': 18})
# plt.rcParams['axes.labelsize'] = 18
# plt.rcParams['axes.labelweight'] = 'bold'
# plt.plot(nn_fpr_keras, nn_tpr_keras,marker='.', label='Neural Network')
# plt.xlabel('False Positive Rate') 
# plt.ylabel('True Positive Rate')
# plt.tight_layout()
# plt.savefig('ROC')

In [ ]:
print(roc_x)
print(roc_y)

In [ ]:
y_pred = []
for score in y_score:
    if(score[0] > score[1] and score[0] > score[2]):
        score[0] = 1
        score[1] = 0
        score[2] = 0
        y_pred.append(0)
    if(score[1] > score[0] and score[1] > score[2]):
        score[0] = 0
        score[1] = 1
        score[2] = 0
        y_pred.append(1)
    if(score[2] > score[0] and score[2] > score[1]):
        score[0] = 0
        score[1] = 0
        score[2] = 1
        y_pred.append(2)

print(y_pred)
print(y_true)

In [ ]:
target_names = ['class 0', 'class 1', 'class 2']

print(classification_report(y_true, y_pred, target_names=target_names))

In [ ]:
def specificity(y_true: np.array, y_pred: np.array, classes: set = None):

    if classes is None: # Determine classes from the values
        classes = set(np.concatenate((np.unique(y_true), np.unique(y_pred))))

    specs = []
    sens = []
    acc = []
    for cls in classes:
        # print(cls)
        y_true_cls = (y_true == cls).astype(int)
        y_pred_cls = (y_pred == cls).astype(int)
        
        # print(y_pred_cls)        
        
        fp = sum(y_pred_cls[y_true_cls != 1])
        tn = sum(y_pred_cls[y_true_cls == 0] == False)
        
        fn = sum(y_pred_cls[y_true_cls != 0]== False)
        tp = sum(y_pred_cls[y_true_cls == 1])
        
        # print(tp,fp,tn,fn)
        
        specificity_val = tn / (tn + fp)
        sens_val = tp / (tp+fn)
        acc_val = (tp + tn) / (tp+fp+tn+fn)
        print('acc')
        print(acc_val)
        print('specificity')
        print(specificity_val)
        print('sens')
        print(sens_val)
        acc.append(acc_val)
        specs.append(specificity_val)
        sens.append(sens_val)
        
    return  np.mean(acc), np.mean(sens),np.mean(specs) 

In [ ]:
specificity(y_true,y_pred)